In [ ]:
from pathlib import Path
import pandas
base_path = Path() / ".." / "results"/ "input" / "capital-stocks"

In [ ]:
icsd = pandas.read_csv(base_path / "icsd.csv")
icsd.tail(2)

In [ ]:
pwt = pandas.read_csv(base_path / "pwt.csv")
pwt.tail(2)

In [ ]:
cwon = pandas.read_csv(base_path / "CWON2021.csv")
cwon.tail(2)

In [ ]:
bem = pandas.read_csv(base_path / ".." / "giri" / "bem_5x5_valfis_adm0.csv")

In [ ]:
df = bem.set_index("GID_0")
df = df.join(icsd.query('year == 2018').drop(columns=['year', 'country']).set_index('isocode'))
df = df.join(cwon.query('year == 2018').drop(columns=['year', 'wb_name', 'unit']).set_index('wb_code'))
df = df.join(pwt.query('year == 2018').drop(columns=['year', 'country']).set_index('countrycode'))
# df = df.dropna()
df.kppp_rppp = df.kppp_rppp.fillna(0)

df["giri__valfis"] = (df.res_sum + df.nres_sum) * 1e-9 # convert unit to billion
df["cwon__pk"] = df.pk * 1e-9 # convert unit to billion
df["icsd__k"] = (df.kgov_rppp + df.kpriv_rppp + df.kppp_rppp)  # assume in billions
df["pwt__cn"] = df.cn * 1e-3 # convert from million to billion

df.loc[["THA", "PHL", "VNM", "IDN"]][["giri__valfis","cwon__pk","icsd__k","pwt__cn"]].plot(kind="bar")


In [ ]:
df.shape

In [ ]:
df_subset = df.dropna(subset=["giri__valfis", "cwon__pk", "icsd__k", "pwt__cn"]).copy()
df_subset.shape

In [ ]:
df_subset["GIRI / CWON"] = (df_subset.giri__valfis / df_subset.cwon__pk)
df_subset["GIRI / ICSD"] = (df_subset.giri__valfis / df_subset.icsd__k)
df_subset["GIRI / PWT"] = (df_subset.giri__valfis / df_subset.pwt__cn)

df_subset["CWON / ICSD"] = (df_subset.cwon__pk / df_subset.icsd__k)
df_subset["CWON / PWT"] = (df_subset.cwon__pk / df_subset.pwt__cn)

ax = df_subset[["GIRI / CWON", "GIRI / ICSD", "GIRI / PWT", "CWON / ICSD", "CWON / PWT"]].plot.box(ylabel="Ratio")
ax.set_title(f"Ratios of Capital Stock Estimates, 2018, n={df_subset.shape[0]} countries")